<a href="https://www.kaggle.com/code/nandakishorejoshi/datefruit-catboost-with-hyperopt?scriptVersionId=95655064" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

> **Using CATBOOST for MultiCalss prediction along with automatic parameter tuning with Hyperopt.**

In this notebook, we will use Catboost to make the multiclass prediction. Instead of manually finding the best hyperparameters for the model, we will employ Hyperopt to do it for us. 
We will exampine our results at the end to understand the performace of Catboost along with hyperopt

In [ ]:
import numpy as np 
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix,mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

import catboost as ctb
from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [2]:
!pip install openpyxl # to read excel file

data_path = "../input/date-fruit-datasets/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx"
data=pd.read_excel(data_path)

df = data.copy()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 3.2 MB/s eta 0:00:00a 0:00:01


In [3]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,...,898.000000,898.000000,898.000000,898.000000,8.980000e+02,8.980000e+02,8.980000e+02,898.000000,898.000000,898.000000
mean,298295.207127,2057.660953,750.811994,495.872785,0.737468,604.577938,0.981840,303845.592428,0.736267,2.131102,...,0.250518,4.247845,5.110894,3.780928,-3.185021e+10,-2.901860e+10,-2.771876e+10,50.082888,48.805681,48.098393
std,107245.205337,410.012459,144.059326,114.268917,0.088727,119.593888,0.018157,108815.656947,0.053745,17.820778,...,0.632918,2.892357,3.745463,2.049831,2.037241e+10,1.712952e+10,1.484137e+10,16.063125,14.125911,10.813862
min,1987.000000,911.828000,336.722700,2.283200,0.344800,50.298400,0.836600,2257.000000,0.512300,1.065300,...,-1.029100,1.708200,1.607600,1.767200,-1.091224e+11,-9.261697e+10,-8.747177e+10,15.191100,20.524700,22.130000
25%,206948.000000,1726.091500,641.068650,404.684375,0.685625,513.317075,0.978825,210022.750000,0.705875,1.373725,...,-0.196950,2.536625,2.508850,2.577275,-4.429444e+10,-3.894638e+10,-3.564534e+10,38.224425,38.654525,39.250725
50%,319833.000000,2196.345450,791.363400,495.054850,0.754700,638.140950,0.987300,327207.000000,0.746950,1.524150,...,0.135550,3.069800,3.127800,3.080700,-2.826156e+10,-2.620990e+10,-2.392928e+10,53.841300,50.337800,49.614100
75%,382573.000000,2389.716575,858.633750,589.031700,0.802150,697.930525,0.991800,388804.000000,0.775850,1.674750,...,0.593950,4.449850,7.320400,4.283125,-1.460482e+10,-1.433105e+10,-1.660367e+10,63.063350,59.573600,56.666675
max,546063.000000,2811.997100,1222.723000,766.453600,1.000000,833.827900,0.997400,552598.000000,0.856200,535.525700,...,3.092300,26.171100,26.736700,32.249500,-1.627316e+08,-5.627727e+08,-4.370435e+08,79.828900,83.064900,74.104600


In [4]:
df.dtypes

AREA               int64
PERIMETER        float64
MAJOR_AXIS       float64
MINOR_AXIS       float64
ECCENTRICITY     float64
EQDIASQ          float64
SOLIDITY         float64
CONVEX_AREA        int64
EXTENT           float64
ASPECT_RATIO     float64
ROUNDNESS        float64
COMPACTNESS      float64
SHAPEFACTOR_1    float64
SHAPEFACTOR_2    float64
SHAPEFACTOR_3    float64
SHAPEFACTOR_4    float64
MeanRR           float64
MeanRG           float64
MeanRB           float64
StdDevRR         float64
StdDevRG         float64
StdDevRB         float64
SkewRR           float64
SkewRG           float64
SkewRB           float64
KurtosisRR       float64
KurtosisRG       float64
KurtosisRB       float64
EntropyRR          int64
EntropyRG          int64
EntropyRB          int64
ALLdaub4RR       float64
ALLdaub4RG       float64
ALLdaub4RB       float64
Class             object
dtype: object

In [5]:
#checking class distribution
df['Class'].value_counts()*100/df.shape[0]

DOKOL     22.717149
SAFAVI    22.160356
ROTANA    18.485523
DEGLET    10.913140
SOGAY     10.467706
IRAQI      8.017817
BERHI      7.238307
Name: Class, dtype: float64

In [6]:
le = LabelEncoder()
df['label'] = le.fit_transform(df.Class.values)

In [7]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class,label
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI,0
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI,0
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI,0
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI,0
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI,0


In [8]:
#checking for nulls
df.isnull().sum()

AREA             0
PERIMETER        0
MAJOR_AXIS       0
MINOR_AXIS       0
ECCENTRICITY     0
EQDIASQ          0
SOLIDITY         0
CONVEX_AREA      0
EXTENT           0
ASPECT_RATIO     0
ROUNDNESS        0
COMPACTNESS      0
SHAPEFACTOR_1    0
SHAPEFACTOR_2    0
SHAPEFACTOR_3    0
SHAPEFACTOR_4    0
MeanRR           0
MeanRG           0
MeanRB           0
StdDevRR         0
StdDevRG         0
StdDevRB         0
SkewRR           0
SkewRG           0
SkewRB           0
KurtosisRR       0
KurtosisRG       0
KurtosisRB       0
EntropyRR        0
EntropyRG        0
EntropyRB        0
ALLdaub4RR       0
ALLdaub4RG       0
ALLdaub4RB       0
Class            0
label            0
dtype: int64

In [9]:
#defining X and Y data 
X,Y=df.drop(['Class','label'],axis=1),df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

In [11]:
print(f"""
X_train shape: {X_train.shape}
X_test shape: {X_test.shape}
y_train shape: {y_train.shape}
y_test shape: {y_test.shape}
""")


X_train shape: (718, 34)
X_test shape: (180, 34)
y_train shape: (718,)
y_test shape: (180,)



In [12]:
X_train=X_train.astype('float')
X_test=X_test.astype('float')

In [13]:
categorical_features_indices = np.where(X_train.dtypes == np.object)[0]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [14]:
import sklearn

In [15]:
#define parameter range
learning_rate=np.linspace(0.01,0.1,10)
max_depth=np.arange(2, 18, 2)
colsample_bylevel=np.arange(0.3, 0.8, 0.1)
iterations=np.arange(50, 1000, 50)
l2_leaf_reg=np.arange(0,10)
bagging_temperature=np.arange(0,100,10)

In [16]:
#Define parameter space, fit conditions, loss function
ctb_reg_params = {
    'learning_rate':     hp.choice('learning_rate',    learning_rate),
    'max_depth':         hp.choice('max_depth',         max_depth),
    #'colsample_bylevel': hp.choice('colsample_bylevel', colsample_bylevel),
    'iterations':       hp.choice('iterations',            iterations),
    #'l2_leaf_reg':       hp.choice('l2_leaf_reg',            l2_leaf_reg),
    #'bagging_temperature':       hp.choice('bagging_temperature',            bagging_temperature),
    'loss_function':       'MultiClass',
    
}
ctb_fit_params = {
    'early_stopping_rounds': 5,
    'verbose': False,
    'cat_features': categorical_features_indices
}
ctb_para = dict()
ctb_para['reg_params'] = ctb_reg_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: mean_absolute_error(y, pred)

In [17]:
#define Hyperopt class
class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            print('entering fmin')
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result

    def ctb_reg(self, para):
        reg = ctb.CatBoostClassifier(**para['reg_params'])
        print('ctb initialized')
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        print('fitting model')
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        print('model fitted')
        pred = reg.predict(self.x_test)
        #loss = para['loss_func'](self.y_test, pred)
        f1=sklearn.metrics.f1_score(self.y_test,pred,average='micro')
        f1=f1*(-1)
        print(f1)
        return {'loss': f1, 'status': STATUS_OK}

In [18]:
#define objective and find the best hyperparameters

obj = HPOpt(X_train, X_test, y_train, y_test)
ctb_opt = obj.process(fn_name='ctb_reg', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=2)

entering fmin
ctb initialized                                      
fitting model                                        
model fitted                                         
-0.9333333333333333                                  
ctb initialized                                                                 
fitting model                                                                   
model fitted                                                                    
-0.9111111111111111                                                             
100%|██████████| 2/2 [00:04<00:00,  2.11s/trial, best loss: -0.9333333333333333]


In [19]:
ctb_opt

{'iterations': 13, 'learning_rate': 3, 'max_depth': 0}

In [20]:
#Save best parametrs in a dictionary
best_param={}
best_param['learning_rate']=learning_rate[ctb_opt['learning_rate']]
#best_param['colsample_bylevel']=colsample_bylevel[ctb_opt['colsample_bylevel']]
best_param['iterations']=iterations[ctb_opt['iterations']]
best_param['max_depth']=max_depth[ctb_opt['max_depth']]
#best_param['l2_leaf_reg']=l2_leaf_reg[ctb_opt['l2_leaf_reg']]
#best_param['bagging_temperature']=bagging_temperature[ctb_opt['bagging_temperature']]


In [21]:
best_param

{'learning_rate': 0.04000000000000001, 'iterations': 700, 'max_depth': 2}

In [22]:
#build model with the best hyperparameters
model=ctb.CatBoostClassifier(iterations=best_param['iterations'], 
                             depth=best_param['max_depth'],
                             learning_rate=best_param['learning_rate'],
                            loss_function='MultiClass',
                            random_seed=42
                            )

In [23]:
model.fit(X_train,y_train,cat_features=categorical_features_indices, eval_set=None, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.8738718	total: 2.96ms	remaining: 2.07s
1:	learn: 1.7987052	total: 5.48ms	remaining: 1.91s
2:	learn: 1.7409211	total: 8.01ms	remaining: 1.86s
3:	learn: 1.6840615	total: 11ms	remaining: 1.91s
4:	learn: 1.6349678	total: 13.6ms	remaining: 1.89s
5:	learn: 1.5901258	total: 16.2ms	remaining: 1.87s
6:	learn: 1.5486967	total: 19.9ms	remaining: 1.97s
7:	learn: 1.5092365	total: 22.5ms	remaining: 1.94s
8:	learn: 1.4743014	total: 25.9ms	remaining: 1.99s
9:	learn: 1.4370383	total: 29.1ms	remaining: 2.01s
10:	learn: 1.4058913	total: 31.6ms	remaining: 1.98s
11:	learn: 1.3760034	total: 34.1ms	remaining: 1.96s
12:	learn: 1.3395104	total: 36.5ms	remaining: 1.93s
13:	learn: 1.3169266	total: 40.2ms	remaining: 1.97s
14:	learn: 1.2808675	total: 43.3ms	remaining: 1.98s
15:	learn: 1.2554333	total: 46.3ms	remaining: 1.98s
16:	learn: 1.2337579	total: 48.8ms	remaining: 1.96s
17:	learn: 1.2089691	total: 52.3ms	remaining: 1.98s
18:	learn: 1.1857311	total: 54.8ms	remaining: 1.96s
19:	learn: 1.1678504	tot

**Verification of the model built**


We will see various metrics from checking confusion matrix to Accuracy of train and test check for overfitting.
We will also see Micro , Macro and Weighted Precision , Recall and F1- Score to understand hte model performance on all class as whole.

In [24]:
train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train,train_pred)
print('Train Accuracy: ', train_acc)
 
test_pred = model.predict(X_test)
test_acc = accuracy_score(y_test,test_pred)
print('Test Accuracy:', test_acc)
y_pred=test_pred

Train Accuracy:  0.9526462395543176
Test Accuracy: 0.9277777777777778


In [26]:
confusion = confusion_matrix(y_test, test_pred)
print('Confusion Matrix\n')
print(confusion)

Confusion Matrix

[[10  0  0  2  0  0  0]
 [ 0 17  1  0  0  0  2]
 [ 0  3 46  0  0  0  1]
 [ 1  0  0  9  0  0  0]
 [ 0  0  0  0 34  0  1]
 [ 0  0  0  0  0 33  0]
 [ 0  2  0  0  0  0 18]]


In [29]:
print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

Micro Precision: 0.93
Micro Recall: 0.93
Micro F1-score: 0.93

Macro Precision: 0.90
Macro Recall: 0.91
Macro F1-score: 0.90

Weighted Precision: 0.93
Weighted Recall: 0.93
Weighted F1-score: 0.93
